In [1]:
import sqlite3

### Basic Setup

In [22]:
conn = sqlite3.connect("local_dbs/test.db")
cursor = conn.cursor()
cursor.execute(
"""
CREATE TABLE IF NOT EXISTS employees (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    age INTEGER,
    department TEXT
)
"""
)

cursor.execute(
    "INSERT INTO employees (name, age, department) VALUES (?, ?, ?)",
    ('John Doe', 30, 'Sales')
)

cursor.execute("SELECT * FROM employees")

rows = cursor.fetchall()

print(rows)

conn.commit()
conn.close()

[(1, 'John Doe', 30, 'Sales')]


### Storing & Retrieving Vectors

In [23]:
import sqlite3
import numpy as np

In [24]:
conn = sqlite3.connect("local_dbs/vector-db.db")
cursor = conn.cursor()

In [27]:
# Create a table for storing vector data
cursor.execute(
"""
CREATE TABLE IF NOT EXISTS vectors (
    id INTEGER PRIMARY KEY,
    vector BLOB NOT NULL
)
"""
)

In [29]:
# Generate some sample vectors
vec1 = np.array([1.2, 3.3, 4.3, 6.4])
vec2 = np.array([1.1, 3.5, 6.3, 2.4])

In [44]:
# Insert vector data into table
cursor.execute(
    "INSERT INTO vectors (vector) VALUES (?)",
    (sqlite3.Binary(vec1.tobytes()),)
)

In [31]:
cursor.execute(
    "INSERT INTO vectors (vector) VALUES (?)",
    (sqlite3.Binary(vec2.tobytes()))
)

In [58]:
cursor.execute("SELECT * from vectors")

In [59]:
rows = cursor.fetchall()

In [60]:
rows

[(1, b'333333\xf3?ffffff\n@333333\x11@\x9a\x99\x99\x99\x99\x99\x19@'),
 (2,
  b'\x9a\x99\x99\x99\x99\x99\xf1?\x00\x00\x00\x00\x00\x00\x0c@333333\x19@333333\x03@'),
 (3, b'333333\xf3?ffffff\n@333333\x11@\x9a\x99\x99\x99\x99\x99\x19@')]

In [53]:
vector = np.frombuffer(rows[1][1], dtype = np.float64)

In [54]:
vector

array([1.1, 3.5, 6.3, 2.4])

In [62]:
vectors = []
for row in rows:
    vector = np.frombuffer(row[1], dtype = np.float64)
    vectors.append(vector)

In [63]:
vectors

[array([1.2, 3.3, 4.3, 6.4]),
 array([1.1, 3.5, 6.3, 2.4]),
 array([1.2, 3.3, 4.3, 6.4])]

### Vector Similarity Search (VSS)

In [64]:
query_vect = np.array([1.0, 3.2, 2.0, 0.5])

In [74]:
cursor.execute(
"""
    SELECT vector from vectors
    ORDER by abs(vector - ?)
    ASC
""", (sqlite3.Binary(query_vect.tobytes()),)
)

In [75]:
res = cursor.fetchone()

In [79]:
closest_vect = np.frombuffer(res[0], dtype = np.float64)

In [80]:
closest_vect

array([1.1, 3.5, 6.3, 2.4])